<a href="https://colab.research.google.com/github/Jun-629/20MA573/blob/master/bsm_calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Volatility calibration to BSM

## Abstract

- Goal
    - Calibrate BSM model for makert price of options


## Problem

All of pricing models require a set of model parameters in order to fully define the dynamics of each model. The process of adjusting model parameters such that the model prices are compatible with market prices is called **Calibration**. 

Input:

- market prices of several options with different strikes and maturities
- spot price
- interest rates

Output:

- Find volatility of BSM model

## Anal


Suppose
- Model prameter $\theta = (\theta_1, ... \theta_m)$ is a vector
- Prices of $n$ instruments are aviable in the market with its market price of $i$th instrument quoted by $C_i$. 
Let's denote $C = (C_1, ..., C_n)$ as a market price vector.
- In theory, $C_i^\theta$ is the model price of the $i$th instrument, and they are denoted as a vector $C^\theta = (C_1^\theta, ...,  C^\theta_n)$. 

Given a distance function $H: \mathbb R^n \times \mathbb R^n \mapsto \mathbb R^+$, calibration is to find the parameter $\theta^*$  which minimize the objective function $H(C^\theta, C)$, i.e.
$$ \theta^* = \arg\min_\theta H(C^\theta, C).$$

Commonly used distance functions for $H$ are 
- $H(x, y) = (\sum_{i=1}^n w_i |x_i - y_i|^p)^{1/p}.$
- $H(x, y) = (\sum_{i=1}^n w_i | \frac{x_i - y_i}{y_i}|^p)^{1/p}.$ If $p = 2$, $w_i =1$, then $H^2$ is called as SSRE (sum of squred relative errors)
- $H(x, y) = (\sum_{i=1}^n w_i |\ln x - \ln y|^p)^{1/p}.$

In this below, we are goint to use, the first case of $H$ with $p = 2$ and $w_i = 1$ for all i's.

# Hw Tasks

## Task-1

- Upload data from [here](https://github.com/songqsh/20s_ma573/blob/master/src/20optiondata2.dat) to your cloud platform
- Calibration/Performance:
  - Filter out 2-mon options and calibrate volatility using the distance function
$$H(x, y) = (\sum_{i=1}^n w_i |x_i - y_i|^2)^{1/2}.$$
  - Calibrated price for option means the computed option price using calibrated volatility and other parameters given by the market. Compare the market price and calibrated price using a plot of two curves: strike vs market (calibrated) price
- Do the same thing for the filter from 5-mon options.
- Which calibrated volatility is bigger, 2-mon or 5-mon?


## Task-2 (optional)
Please discuss the validity of the following statements:
- Given an underlying asset, options with shorter maturity can be better fitted than a longer maturity.
- Suppose Assest1(ex. AAPL) is more liquid than Asset2 (ex. TAL). Then Asset1 can be fit better than Asset2.

In [0]:
%cd~

!git clone https://github.com/Jun-629/20MA573.git 
pass

/root
fatal: destination path '20MA573' already exists and is not an empty directory.


In [0]:
%cd 20MA573/src/
%ls

from bsm import *

/root/20MA573/src
bsm.py
hw1_grid_random_walk.ipynb
Hw2_finite_difference.ipynb
Hw2_pdf.ipynb
Hw3_bsm_price_change.ipynb
Hw3_Explicit_bsm_greeks.ipynb
Hw3_implied_volatility.ipynb
Hw3_payoff_structure_of_option_combinations.ipynb
Hw4_Monotonicity_in_volatility.ipynb
optiondata.dat
Project_1.ipynb
Project_2.ipynb


In [0]:
np_option_data1 = np.loadtxt('optiondata.dat', comments='#', delimiter=',')

print('The data of 2-mon options will be shown as follows:')
print('>>>otype,         maturity, strike,       option_price')
print(np_option_data1[0:4])

The data of 2-mon options will be shown as follows:
>>>otype,         maturity, strike,       option_price
[[  1.           0.16666667  97.           5.32705461]
 [  1.           0.16666667  99.           3.86224255]
 [  1.           0.16666667 101.           2.7204371 ]
 [  1.           0.16666667 103.           2.1202793 ]]


In [0]:
num_row = np_option_data1.shape[0]
option_list = []

for i in range(num_row):
  option1 = VanillaOption(
      otype = np_option_data1[i,0],
      strike = np_option_data1[i,2],
      maturity = np_option_data1[i,1],
      market_price = np_option_data1[i,3]
  )   
  option_list.append(option1)

gbm = Gbm( init_state = 100., drift_ratio = .05, vol_ratio = .1)  

#expand one column for vol
np_option_data2 = np.append(np_option_data1, np.zeros([num_row,1]), axis = 1)
#compute implied vols and add them into the last column
for i in range(num_row):
  np_option_data2[i,4] = implied_volatility(gbm, option_list[i])

IV = np_option_data2[0:8,4]
IV_2mon = np_option_data2[0:4,4]
IV_5mon = np_option_data2[4:8,4]

model_vol = np.arange(0.001, 0.1, 0.0005)
 
# Calculate different B-S price
BS_price_list = []
for i in range(num_row):
  BS_price = Gbm(100., 0.05, IV[i]).bsm_price(option_list[i])
  BS_price_list.append(BS_price)
# print('The ' + str(i+1) + 'th B-S price under the corresponding implied volatility is ' + str(BS_price))

a = 0
b = 0
c = 0
for i in range(4):
  a = (abs(np_option_data1[0:4,i] - BS_price_list[i]))**2
  b = b + a
  c = np.

from scipy.optimize import fsolve

def f(x):
  gbm2 = Gbm(vol_ratio = x)
  option2 = VanillaOption(otype=1)
  return(gbm2.bsm_price(option2)-10)

ans_sig_2 = fsolve(f,0.1)
print('>>> The implied volatility is ' + str(ans_sig_2))



[0.19125    0.17724609 0.17183594 0.18681641 0.170625   0.16804688
 0.18373047 0.20901367]
[5.326527938668505, 3.8623905475124616, 2.7201676263810484, 2.120145596985431, 7.237365128554927, 5.951468640679728, 5.26395723599834, 4.975020170319965]
>>> The implied volatility is [0.30199229]


In [0]:

a
np.sqrt(4)

2.0